# Welcome to the Agent Exploration Colab! 🎯🤖

This Colab notebook is designed to introduce the fundamental components of an agent, including **prompts, tools, and callbacks**. We will explore how agents interact and how multiple agents can work together efficiently.


By the end of this notebook, you will have a clear understanding of:

✅ The essential building blocks of an agent

✅ How to enhance agent capabilities using tools and callbacks

✅ The fundamentals of multi-agent collaboration



Let’s dive in and start building intelligent agents! 🚀

## Prerequisites

You are probably excited to start already, but before that, there are some
prerequisites you need to prepare. (Sorry!)

### Install Synthora and other packages

Synthora requires Python 3.10+. You can install Synthora via pip:

In [ ]:
%pip install synthora=="0.1.10"
%pip install wikipedia
%pip install googlesearch-python
%pip install trafilatura
%pip install bs4
%pip install "rich[jupyter]"
%pip install python-pptx
%pip install requests
%pip install duckduckgo_search

In [1]:
import os
from getpass import getpass

from synthora.agents import VanillaAgent, ReactAgent
from synthora.prompts import BasePrompt
from synthora.prompts.buildin import ZeroShotCoTPrompt, ZeroShotReactPrompt
from synthora.callbacks.output_handler import OutputHandler
from synthora.callbacks import RichOutputHandler
from synthora.toolkits.file_toolkit import FileToolkit
from synthora.toolkits.search_toolkit import SearchToolkit
from synthora.toolkits.webpage_toolkit import TrafilaturaWebpageReader
from synthora.toolkits.slides import SlidesToolkit
from synthora.toolkits.decorators import tool
from synthora.models import OpenAIChatBackend
from synthora.messages import user
from typing import Dict, List

from synthora.workflows import task
from synthora.workflows.scheduler.thread_pool import ThreadPoolScheduler
from synthora.toolkits.search_toolkits import search_wikipedia
from pydantic import BaseModel
import json

## The Difference Between LLMs and Agents 🤖

### Setup your API Key

In this tutorial, we will use OpenAI as our model provider.

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key here: ")

Next, let's define an OpenAI Model and have a try!

In [109]:
model = OpenAIChatBackend(api_key=os.environ["OPENAI_API_KEY"], model_type="gpt-4o")

You can use function `user` to define a message from user!

In [110]:
message = user("hello, my name is Tom.")
message.to_openai_message()

{'content': 'hello, my name is Tom.', 'role': 'user', 'name': 'user'}

Imagine you're at a party. An LLM is like someone with amazing knowledge but terrible memory - they'll forget your name right after you tell them! 

Let me show you what I mean:

In [111]:
print(model.run([message]).content)
# Hello Tom! How can I assist you today?

print(model.run([user("What is my name?")]).content)
# I'm sorry, but I don't have access to personal data about individuals unless it's shared with me during our conversation. 

'Hello Tom! How can I assist you today?'

That's because llm model is stateless, it will not save our previous conversation.

If we want model to have previous context, we need to provide them manully.

In [ ]:
model.run([message, user("What is my name?")]).content

But an Agent? They're like a person with both knowledge AND memory! 

They'll remember your name throughout the conversation:

In [ ]:
agent = VanillaAgent.default()
agent.run("Hello, my name is Tom, how are you today?")
resp = agent.run("What is my name?")
print(resp.unwrap().content)

Unlike the stateless LLM model, agents maintain state and can remember information from previous interactions. As we can see, when we ask the agent 'What is my name?' after introducing ourselves, it correctly remembers that our name is Tom. This is because agents maintain conversation history and context between interactions, allowing them to reference and use previously shared information.

This stateful nature makes agents more suitable for ongoing conversations and tasks that require memory of past interactions. The agent doesn't need to be explicitly provided with the conversation history each time - it automatically maintains this context internally."


## Understanding Prompts: The Core of Agent Behavior 🎯

Prompts are the foundation of how agents think and behave. They act as instructions or guidelines that shape how an agent interprets and responds to input. Let's explore how prompts work and how we can customize them to create agents with specific behaviors.



Let's look at a simple example to understand how different prompts affect agent behavior. 

We'll ask agents to count the number of 'p's in the word 'applepiepp'.

First, with a default agent:

In [ ]:
resp = agent.run("How many 'p's are in the word applepiepp")
print(resp.unwrap().content)


The agent might give an incorrect answer because it's making a quick judgment without breaking down the problem.

Now, let's use an agent with Chain-of-Thought prompting:

In [ ]:
agent = VanillaAgent.default(ZeroShotCoTPrompt)
resp = agent.run("How many 'p's are in the word applepiepp")
print(resp.unwrap().content)


With Chain-of-Thought prompting, we see a significant improvement in accuracy because the agent:

1. Systematically breaks down the word: 'a-p-p-l-e-p-i-e-p-p'
2. Identifies and counts each 'p': p1, p2, p3, p4, p5
3. Arrives at the correct answer: 5 p's

This demonstrates how prompting strategies can significantly impact an agent's accuracy. By encouraging step-by-step reasoning through Chain-of-Thought prompting, we reduce errors and get more reliable results. The ZeroShotCoTPrompt helps the agent think more carefully and show its work, leading to better problem-solving."


### Key Takeaways About Prompts 🔑

From this simple counting example, we can draw several important conclusions about prompts:

1. **Impact on Accuracy**: The right prompt can significantly improve an agent's accuracy. While the default agent made counting errors, the Chain-of-Thought prompt led to correct results.

2. **Reasoning Process**: Prompts don't just affect the final answer - they shape how an agent approaches problem-solving. 

3. **Task Suitability**: Even for seemingly simple tasks like counting letters, the choice of prompt can make a significant difference in performance. This suggests that for more complex tasks, careful prompt design becomes even more crucial.

Understanding these aspects of prompts is essential for developing effective AI applications, as the right prompt can be the difference between success and failure in agent-based tasks.



The `Prompt` functionality in `synthora` provides a way to create and manage customizable prompt templates with dynamic argument formatting.

This system integrates with agents like `VanillaAgent` and allows for flexible prompt handling during runtime.


In [ ]:
prompt = "You are an AI Assistan. Your name is {name}"

In [ ]:
try:
    agent = VanillaAgent.default(prompt=prompt)
    resp = agent.run("What is your name?")
except Exception as e:
    print(e)

Oops, it seems the agent is not able to use the prompt.

That's because in our prompt, we have a placeholder `{name}`, which should be replaced with the actual name.

To fix this, we can provide the `name` argument to the agent:

In [ ]:
resp = agent.run("What is your name?", name="Tom")
print(resp.unwrap().content)

## Callback: A Powerful Tool for Monitoring and Logging an Agent 🔍

Unlike LLM Model, agent provides a higher level of abstraction, which makes it more difficult to understand what's happening inside the agent.

To monitor and log the agent's behavior, we can use callbacks, which provide a powerful way to log and track events throughout your agent's workflow.

---

### What Are Callbacks?

Callbacks act as event listeners, providing updates on various stages of your agent's operations. They notify you when:

- The agent starts processing.
- A tool is utilized.
- Decisions are made.
- An error occurs.
- ...

To add a callback to an agent, we can simply pass a list of callbacks to the agent's constructor:

In [ ]:
agent = ReactAgent.default(prompt=ZeroShotReactPrompt, handlers=[RichOutputHandler()])
resp = agent.run("How many 'p's are in the word applepiepp")

In this example, we use `RichOutputHandler` as a callback, which will print the agent's response in a rich format.

If you provide a list of callbacks, the agent will execute them in the order you provided.

## Tools: Adding External Capabilities to Agents 🛠️

Most of the time, agent is a powerful tool that can be used to solve various problems. 

However, sometimes, they are not enough, so we need to add some external tools to improve the agent's performance.


But, how can an agent use tools? How can the agent know how to use the tool?

This happens at the model level.

To let the model know how to use the tool, we need to provide the tool's description and signature to the model.
Including the tool's name, description, and parameters.

This can be very complex, that's why we provide a `tool` decorator to help you get the tool's description and signature.

> What is `decorator` in python?

> Decorator is a function that modifies the behavior of another function.

> It is often used to wrap the original function with some additional functionality.

> For example, you can use it to log the function's input and output.


In [ ]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

After you decorate the function with `@tool`, you can get the tool's description and signature by calling `tool.schema`.

In [ ]:
print(json.dumps(multiply.schema, indent=4))

`Synthora` will automatically add the tool's description and signature to the model when you pass the tool to the agent.

Now, let's see how to use the tool in an agent and how the tools can improve the agent's performance.

In [ ]:
agent = VanillaAgent.default()
resp = agent.run("8743687 * 23478 = ?")
print(resp.unwrap().content)

In [2]:
8743687 * 23478

205284283386

Oops, it's very close, but not exactly correct.

Now, let's add the tool to the agent:

In [ ]:
agent = VanillaAgent.default(tools=[multiply], handlers=[OutputHandler()])
resp = agent.run("8743687 * 23478 = ?")
print(resp.unwrap().content)

As you can see, the agent can now use the tool to solve the problem and get the correct answer!

Now, let's add another tool to the agent:

In [ ]:
agent = VanillaAgent.default(tools=[multiply, add], handlers=[OutputHandler()])
resp = agent.run("8743687 * 23478 + 983479 = ?")
print(resp.unwrap().content)

In [ ]:
8743687 * 23478 + 983479

Now we can move on to more complex tasks, a real world example.

We will provide a set of tools to the agent, and let the agent can search the web, read the webpage, and generate a slides for us.

It's not hard! Synthora has already provided a set of tools for you, you just need to add them to your agent!

In [ ]:
agent = VanillaAgent.default(
    tools=SlidesToolkit(api_key=getpass("Your UNSPLASH_API_KEY")).sync_tools + [*TrafilaturaWebpageReader(full_text=True).sync_tools],
    handlers=[OutputHandler()],
)

In [ ]:
resp = agent.run("generate a PPT about https://ai4ocean.xyz, including Products, Team, Research, Projects, Publications, Education, Partnership, etc. ")
print(resp.unwrap().content)

If everything goes well, you can see the agent uses multiple tools multiple times to solve the problem.

And you will get a PPT about the given topic under `cache` folder.

Is it cool?

## Multi-Agent Collaboration: When One is Not Enough 🤝

In many cases, a single agent may not be sufficient to solve a complex problem. This is where multi-agent collaboration comes in.

By combining the strengths of multiple agents, you can tackle more challenging tasks and achieve better results.

Now, let's see how to use multi-agent collaboration to solve a problem.

We will use two agents to solve a problem:

1. A web search agent to search the web for the best resources.
2. A file operation agent to write the resources to a file.


In [ ]:
search_agent = VanillaAgent.default(
    """You are a web search agent. You are tasked with
    finding the best resources on the web for a given topic.
    You should provide a list of the top 5 resources that you find,
    along with a brief summary of each resource.
    You should also provide a brief summary of the topic itself. """,
    name="WebSearchAgent",
    tools=[SearchToolkit().search_duckduckgo],
)
search_agent.description = """
A web search agent that finds the best resources on the web for a given topic.
"""

In [ ]:
file_operate_agent = VanillaAgent.default(
    """You are an AI assistant that helps users with file operations.""",
    name="FileOperationAssistant",
    tools=[FileToolkit().write_file],
)
file_operate_agent.description = """
An AI assistant that helps users with file operations.
"""

To simplify the multi-agent collaboration, the agent in Synthora can be seen as a `tool` that can be used in other agents.

So, we can simply pass the `search_agent` and `file_operate_agent` to the `supervisor` agent.

The `supervisor` agent will use the `search_agent` to search the web for the best resources, and use the `file_operate_agent` to write the resources to a file.



In [ ]:
supervisor = VanillaAgent.default(
    tools=[search_agent, file_operate_agent],
    handlers=[OutputHandler()],
)

In [ ]:
resp = supervisor.run(
    "Find some llm tutorials for me and write them to a file named llm.md.",
)
print(resp.unwrap().content)

If everything goes well, you can see the `supervisor` agent calls the `search_agent` to search the web and the `file_operate_agent` to write the resources to a file.

And you will get a file named `llm.md` under the current working directory.



## Workflow: Orchestrating Agents with Flexibility

Workflow is a powerful system in Synthora that can be used to orchestrate agents to solve various problems, allowing users to define their own workflows with a high level of flexibility.

In this tutorial, we will explore how to use workflow to manage and automate tasks efficiently. We will start with basic concepts and gradually move to more advanced features. Thanks to the flexibility of workflow, the overall process is simple and can be easily customized.

Now, if you are ready, let's start!

In [88]:

from synthora.workflows import task
from synthora.workflows.base_task import BaseTask
from synthora.workflows.scheduler import ThreadPoolScheduler
from synthora.workflows.scheduler.base import BaseScheduler

### Components of Workflow

A workflow in Synthora is composed of three main components: scheduler, task, and context.

- **Task**: The basic execution unit, usually a function.
- **Scheduler**: Invokes tasks either in parallel or in series according to specific strategies.
- **Context**: Allows tasks to read and store information during execution, and supports advanced features like loops and conditional statements.(Will not mention today)

Similar like `tool`, you can also use a decorator to declare a task.

In [4]:
@task
def add_task(x: int, y: int) -> int:
    return x + y

def add(x: int, y: int) -> int:
    return x + y

You can call the task directly just like a function.

In [5]:
add_task(1, 2)

3

### Task Signature

Task signatures allow you to predefine task parameters, enabling users to implement more complex functionalities.

By using task signatures, you can set default values for the parameters of a task, making it easier to reuse tasks with predefined configurations. 

This is particularly useful when you need to run the same task multiple times with the same parameters or when you want to create more complex workflows by chaining tasks together.

> In one word, signature is a way to predefine the parameters of a function.

In [6]:
add_task.s(1)
# or
# add_task.signature(1)


f6253f4f-8bbf-47cf-8ce5-66b92e4b49e0

In [92]:
try:
    add_task(1, 2)  # will raise an error
except TypeError as e:
    print(e)

add_task() takes 2 positional arguments but 3 were given


Because the task has a signature, which means the task has a default parameter.

This means you only need to provide **ONE** parameter to the task!.

In [93]:
add_task(2)

3

### Serial and Parallel

In Synthora, tasks can be executed either serially or in parallel, providing flexibility in how workflows are structured.


### Serial Tasks

Serial tasks are executed one after another, with each task's output being passed as input to the next task. This allows for a linear flow of data through the tasks.

You can use Python operators to declare serial workflows, which can simplify the creation process:

In [98]:
flow = BaseTask(add) >> BaseTask(add).s(1) >> BaseTask(add).s(2)
flow.run(1, 1)

5

why the output is 5?

> This is because the workflow will add the return value of the previous task as a parameter to the next task.

>First Step: The flow gets the input: (1, 1), and passes it to the first task. The first task returns 1 + 1 = 2.

>For the second task, we have pre-specified the input as 1, which, combined with the return value of the previous task (2), is passed as parameters. The second task returns 2 + 1 = 3.

> Similarly, the third task returns 3 + 2 = 5.

### Parallel Tasks
Parallel tasks are executed simultaneously, with each task receiving the same input parameters. This allows for concurrent processing and can significantly speed up the workflow when tasks are independent of each other.

Parallel tasks can also be declared using expressions:

In [99]:
flow = (
    BaseTask(add).s(0)
    | BaseTask(add).s(1)
    | BaseTask(add).s(2)
    | BaseTask(add).s(3)
)
flow.run(1)

[1, 2, 3, 4]

Unlike serial tasks, the input parameters for parallel tasks are passed to each task individually. For example:

task1 receives parameters 0, 1

task2 receives parameters 1, 1

task3 receives parameters 2, 1

task4 receives parameters 3, 1

Each task will process its own set of parameters independently and simultaneously.

Now, let's move on to a more complex example.

In marine science, it is not enough just using a general LLM model to answer the question.

We need to use a marine expert to answer the question!

By using `Fine-Tuning`, we can inject the marine knowledge into the model.

But how can we get the data to train the model?

We can use a `workflow` to generate them!



First, we need to define the data structure we want to generate.

`MarineDataItem` is the data structure we want to generate.

It contains a question and an answer.

`MarineData` is a list of `MarineDataItem`.



In [100]:
class MarineDataItem(BaseModel):
    question: str
    answer: str

class MarineData(BaseModel):
    items: List[MarineDataItem]

Next, we need to define the task we want to use.

`search_wikipedia_task` is a task that searches the web for the marine knowledge.

`generate_data` is a task that generates the data by using these marine knowledge.



In [103]:
@task
def search_wikipedia_task(concepts: str) -> str:
    try:
        return search_wikipedia.run(concepts).unwrap()
    except Exception as e:
        return str(e)

In [102]:
@task
def generate_data(doc: str) -> MarineData:
    agent = VanillaAgent.default("You are a marine data expert. You are given a document and you need to generate several questions and answers about the document.")
    agent.model.config["response_format"] = MarineData
    results = agent.run(doc).unwrap().parsed
    return results

Then, we need to define the concepts we want to search and generate the data.

In [104]:
concepts = [
    "Acoustic seabed classification",
    "Advection",
    # "Ageostrophy",
    # "Baroclinity",
    # "Coriolis frequency"
]

Now, we can define the workflow.

The first step is to search the web for the marine knowledge.

The second step is to generate the data by using these marine knowledge.


To improve the efficiency, we can use `ThreadPoolScheduler` to run the tasks in parallel.

In [107]:
flow = ThreadPoolScheduler.map(search_wikipedia_task >> generate_data, concepts)
datas = flow.run()

In [108]:
for data in datas:
  for item in data.items:
    print(item.question)
    print(item.answer)
    print('-' * 30)


What is acoustic seabed classification?
Acoustic seabed classification is the partitioning of a seabed acoustic image into discrete physical entities or classes. It is used to characterize the seabed and its habitats by linking the classified regions to their physical, geological, chemical, or biological properties.
------------------------------
What are the two main categories of seabed classification based on acoustic properties?
The two main categories of seabed classification based on acoustic properties are surficial seabed classification and sub-surface seabed classification.
------------------------------
What is the focus of surficial seabed classification?
Surficial seabed classification is primarily concerned with distinguishing the marine benthic habitat characteristics such as hard, soft, rough, smooth, mud, sand, clay, and cobble of the surveyed area.
------------------------------
Which technologies are commonly used for surficial seabed classification?
The most commonly